#### all imports

In [ ]:
import tensorflow as tf
import os
import numpy as np
import json
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Embedding, LSTM, Dense, Dropout, concatenate
from keras.models import Model
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import pickle

#### read all dataset quetions and answers from json files

In [ ]:
train_file_questions = '/kaggle/input/visual-question-answering/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json'
train_file_annotations = '/kaggle/input/visual-question-answering/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json'
val_file_questions = '/kaggle/input/visual-question-answering/v2_Questions_Val_mscoco/v2_OpenEnded_mscoco_val2014_questions.json'
val_file_annotations = '/kaggle/input/visual-question-answering/v2_Annotations_Val_mscoco/v2_mscoco_val2014_annotations.json'
test_file_questions = '/kaggle/input/visual-question-answering/v2_Questions_Test_mscoco/v2_OpenEnded_mscoco_test2015_questions.json'

with open(train_file_questions, 'r') as f:
    train_questions = json.load(f)['questions']
    f.close()

with open(train_file_annotations, 'r') as f:
    train_annotations = json.load(f)['annotations']
    f.close()

with open(val_file_questions, 'r') as f:
    val_questions = json.load(f)['questions']
    f.close()

with open(val_file_annotations, 'r') as f:
    val_annotations = json.load(f)['annotations']
    f.close()

#with open(test_file_questions, 'r') as f:
#    test_questions = json.load(f)['questions']
#    f.close()

#### read all train and validate image features with IDs from the pkl files

In [ ]:
# Read dictionary pkl file
with open('/kaggle/input/featured/train_image_feature.pkl', 'rb') as fp:
    train_imgs_features = pickle.load(fp)
    print('successful')
# Read dictionary pkl file
with open('/kaggle/input/featured/val_image_features.pkl', 'rb') as fp:
    val_imgs_features = pickle.load(fp)
    print('successful')

#### append validate to train features

In [ ]:
train_imgs_features.update(val_imgs_features)

In [ ]:
len(train_imgs_features)

#### append validate questions and answers to train questions and answers

In [ ]:
# Combine the training and validation questions and annotations
train_questions += val_questions
train_annotations += val_annotations

#### encode questions and answers, and create image features list 

In [ ]:
# Extract the questions and answers
questions = []
answers = []
features_id = []

for i in range(len(train_questions)):
    questions.append(train_questions[i]['question'])
    answers.append(train_annotations[i]['multiple_choice_answer'])
    features_id.append(train_questions[i]["image_id"])

# Tokenize the questions
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(questions)

max_question_length = 30
padded_sequences = pad_sequences(sequences, maxlen=max_question_length)

# Convert the answers to one-hot vectors
answers_tokenizer = Tokenizer()
answers_tokenizer.fit_on_texts(answers)
answer_word_index = answers_tokenizer.word_index
num_classes = len(answer_word_index)
answer_sequences = answers_tokenizer.texts_to_sequences(answers)

# Pad the answer sequences to ensure they all have the same length
max_answer_length = max(len(seq) for seq in answer_sequences)
padded_answers = pad_sequences(answer_sequences, maxlen=max_answer_length)

# Get the unique answers in the dataset and create a dictionary to map them to integer labels
unique_answers = list(set(answers))
label_map = {answer: i for i, answer in enumerate(unique_answers)}

# Convert the answers to integer labels and then to one-hot vectors
labels = [label_map[answer] for answer in answers]
one_hot_answers = to_categorical(labels, num_classes=len(unique_answers))

#### empty some memory to prevent memory overflow

In [ ]:
del train_file_questions
del train_file_annotations
del val_file_questions
del val_file_annotations
del test_file_questions
del train_questions
del train_annotations
del val_questions
del val_annotations
del val_imgs_features

In [ ]:
del tokenizer
del sequences
del answers_tokenizer
del answer_word_index
del max_answer_length
del padded_answers
del label_map
del labels

In [ ]:
del questions
del answers

#### shape of the dataset

In [ ]:
print(len(features_id))
print(padded_sequences.shape)
print(one_hot_answers.shape)

### split train into 70% train and 30% test 
#### (inplace to prevent memory overflow)

In [ ]:
split_indices = np.random.randint(low=0, high=len(features_id), size=int(len(features_id)*0.3))
split_indices = sorted(split_indices,reverse=True) 

In [ ]:
test_padded_sequences = []
padded_sequences = list(padded_sequences)
for i in split_indices:
    test_padded_sequences.append(padded_sequences.pop(i))

In [ ]:
test_one_hot_answers = []
one_hot_answers = list(one_hot_answers)
for i in split_indices:
    test_one_hot_answers.append(one_hot_answers.pop(i))

In [ ]:
test_features_id = []
for i in split_indices:
    test_features_id.append(features_id.pop(i))

### split 30% test into 20% test and 10% validate
#### (inplace to prevent memory overflow)

In [ ]:
split_indices = np.random.randint(low=0, high=len(test_features_id), size=int(len(test_features_id)*0.3))
split_indices = sorted(split_indices,reverse=True)

In [ ]:
val_padded_sequences = []
for i in split_indices:
    val_padded_sequences.append(test_padded_sequences.pop(i))

In [ ]:
val_one_hot_answers = []
for i in split_indices:
    val_one_hot_answers.append(test_one_hot_answers.pop(i))

In [ ]:
val_features_id = []
for i in split_indices:
    val_features_id.append(test_features_id.pop(i))

#### Second Model: GRU for questions and concat with inception v3 output
##### uses nadam optimizer

In [ ]:
# Define the input layers
question_input = Input(shape=(max_question_length, ), name='question_input')
image_input = Input(shape=(2048, ), name='image_input')

# Define the embedding layer for the questions
question_embedding = Embedding(input_dim=len(word_index)+1, output_dim=300, input_length=max_question_length,
                               name='question_embedding')(question_input)

# Define the LSTM layer for the questions
question_lstm = tf.keras.layers.GRU(units=256, name='question_lstm',return_sequences = True)(question_embedding)
question_lstm = Dropout(0.2, name='question_dropout')(question_lstm)
question_lstm2 = tf.keras.layers.GRU(units=256, name='question_lstm2')(question_lstm)
question_lstm2 = Dropout(0.2, name='question_dropout2')(question_lstm2)

# Define the dense layer for the image features
image_dense = Dense(units=256, activation='relu', name='image_dense')(image_input)
image_dense = Dropout(0.2, name='image_dropout')(image_dense)

# Concatenate the output from the LSTM and dense layers
dense_1 = concatenate([question_lstm2, image_dense], name='concatenated')
dense_2 = Dense(512, activation='relu')(dense_1)
dense_3 = Dense(256, activation='relu')(dense_2)
# Define the output layer for the classification

output = Dense(units=29332, activation='softmax', name='output')(dense_3)

# Define the model
model = Model(inputs=[question_input, image_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])

In [ ]:
model.summary()

#### creating a custom generator

In [ ]:
def data_generator(image_features, padded_questions, labels, batch_size):
    num_samples = len(labels)
    steps_per_epoch = num_samples // batch_size
    while True:
        for i in range(steps_per_epoch):
            batch_image_features = []
            for j in  image_features[i*batch_size:(i+1)*batch_size]:
                batch_image_features.append(train_imgs_features[j])
            batch_padded_questions = padded_questions[i*batch_size:(i+1)*batch_size]
            batch_labels = labels[i*batch_size:(i+1)*batch_size]
            yield [np.asarray(batch_padded_questions), np.asarray(batch_image_features)], np.asarray(batch_labels)

#### training the model using 70% train and 10% validate

In [ ]:
batch_size = 128
steps_per_epoch = len(one_hot_answers) // batch_size
model.fit(data_generator(features_id,padded_sequences,one_hot_answers,batch_size),
          steps_per_epoch=steps_per_epoch,
          epochs=10,
         validation_data = data_generator(val_features_id,val_padded_sequences,val_one_hot_answers,batch_size),
         validation_steps = int(len(val_features_id)/batch_size))

#### saving the model

In [ ]:
model.save("inc_GRU_Nadam.h5")

#### end